# CNN NEWS ARTICLES SENTIMENT ANALYSIS

**Importing Libraries**

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords

**SENTIMENT ANALYSIS**

**Creating a dictionary of Positive & Negative words**

In [67]:
pos_words = open('MasterDictionary\positive-words.txt','r', encoding="utf8").read()
pos_words = pos_words.replace("\n"," ")
pos_words = pos_words.split(" ")

In [68]:
neg_words = open("MasterDictionary/negative-words.txt",'r').read()
neg_words = neg_words.replace("\n"," ")
neg_words = neg_words.split(" ")

In [69]:
output_data = pd.read_excel("output_data.xlsx")
output_data.head()

,Unnamed: 0,INDEX,URL,TITLE,BODY
0,0,1,https://www.cnn.com/2022/03/13/us/william-hurt...,"William Hurt, actor known for 'Body Heat' and ...","\n William Hurt, the Oscar-winning star o..."
1,1,2,https://www.cnn.com/2022/03/12/us/best-in-us-s...,A grieving surfer is taking hundreds of strang...,"\nEditor’s Note: Two years ago, Covid-19 turne..."
2,2,3,https://www.cnn.com/2021/09/03/us/what-is-texa...,Explainer: What is the Texas abortion ban? - ...,"Jeevan Ravindran, CNNUpdated 1908 GMT (0308 HK..."
3,3,4,https://www.cnn.com/2022/02/24/us/iyw-how-to-h...,How to provide aid to civilians in Ukraine | C...,\n More than a year after Russian forces ...
4,4,5,https://www.cnn.com/2022/03/07/us/iyw-internat...,International Women's Day 2022: She became a r...,\n It was the middle of the night when Bi...


In [70]:
def clean_data(dataset):

    dataset = re.sub('[^a-zA-Z]', ' ', dataset) #To remove non-alphabetical characters
    dataset = re.sub(r'\s+', ' ', dataset) #To remove more than one blank space
    words_dataset = dataset.lower().split(" ")

    stopwords = open('nltk stopwords.txt','r', encoding='utf8').read()
    stopwords = stopwords.split()

    clean_dataset = []

    for word in words_dataset:
        if word not in stopwords:
            clean_dataset.append(word)

    clean_dataset = list(filter(None,clean_dataset))

    return clean_dataset

In [71]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = ["a","e","i","o","u","y"]
    exceptions = ("es","ed","e")
    
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith(exceptions):
            count -= 1
        if count == 0:
            count += 1
    return count


In [72]:
pos_score = []
neg_score = []
pol_score = []
sub_score = []
avg_sen_len = []
percentage_complex_words = []
fog_index = []
avg_words_per_sen = []
count_complex_words = []
word_count = []
syllable_per_word = []
personal_pronouns = []
avg_word_len = []

In [73]:
#Calculation of different scores

for i in range(134):

    content = output_data._get_value(i,'BODY')

    sen_dataset = content.split(".")
    total_sen = len(sen_dataset) #Counting total no of sentences

    clean_dataset = clean_data(content)
    total_words = len(clean_dataset)
   
    positive_score = 0
    negative_score = 0

    for i in range (total_words):
        if clean_dataset[i] in pos_words:
            positive_score +=1
        elif clean_dataset[i] in neg_words:
            negative_score +=1
    

    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score)/ ((total_words) + 0.000001)

    pos_score.append(round(positive_score))
    neg_score.append(round(negative_score))
    pol_score.append(round(polarity_score,4))
    sub_score.append(round(subjectivity_score,4))


    #Count of personal pronouns
    pronouns = ['I','we','We','my','My','ours','our','Our','us']
    count_pronoun = 0

    #Calculation of average word length
    character_count = 0

    for word in clean_dataset:
        character_count += len(word)
        if word in pronouns:
            count_pronoun += 1

    average_word_len = character_count/total_words
    avg_word_len.append(round(average_word_len))

    personal_pronouns.append(count_pronoun)

    #Word count
    word_count.append(total_words)

    #Calculation of no of complex words & count of syllables per word
    syllables = 0
    s_count = 0
    complex_words = 0

    for i in range(total_words):
        syllables = syllable_count(clean_dataset[i])
        if (syllables>2):
            complex_words += 1

        s_count+=syllables
    
    count_complex_words.append(complex_words)

    syllable_count_per_word = s_count/total_words
    syllable_per_word.append(round(syllable_count_per_word))

    #Calculation of percentage of complex words
    percentage_of_complex_words = (complex_words/total_words)*100
    percentage_complex_words.append(round(percentage_of_complex_words))

    #Calculation of average sentence length
    average_sentence_length = total_words/total_sen
    avg_sen_len.append(round(average_sentence_length))

    #Calculation of fog index
    fog_i = (average_sentence_length+percentage_of_complex_words)*0.4
    fog_index.append(round(fog_i))

   


**ANALYSIS OF READABILITY**


In [74]:
#Entering all the calculated data to the dataframe

output_data['POSITIVE SCORE']=pos_score
output_data['NEGATIVE SCORE']=neg_score
output_data['POLARITY SCORE']=pol_score
output_data['SUBJECTIVITY SCORE']=sub_score
output_data['AVG SENTENCE LENGTH']=avg_sen_len
output_data['PERCENTAGE OF COMPLEX WORDS']=percentage_complex_words
output_data['FOG INDEX']=fog_index
output_data['COMPLEX WORD COUNT']=count_complex_words
output_data['SYLLABLE PER WORD']=syllable_per_word
output_data['WORD COUNT']=word_count
output_data['PERSONAL PRONOUNS']=personal_pronouns
output_data['AVG WORD LENGTH']=avg_word_len

output_data

,Unnamed: 0,INDEX,URL,TITLE,BODY,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,COMPLEX WORD COUNT,SYLLABLE PER WORD,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,1,https://www.cnn.com/2022/03/13/us/william-hurt...,"William Hurt, actor known for 'Body Heat' and ...","\n William Hurt, the Oscar-winning star o...",13,17,-0.1333,0.1852,9,21,12,34,2,162,0,6
1,1,2,https://www.cnn.com/2022/03/12/us/best-in-us-s...,A grieving surfer is taking hundreds of strang...,"\nEditor’s Note: Two years ago, Covid-19 turne...",40,28,0.1765,0.1048,9,14,9,90,1,649,0,6
2,2,3,https://www.cnn.com/2021/09/03/us/what-is-texa...,Explainer: What is the Texas abortion ban? - ...,"Jeevan Ravindran, CNNUpdated 1908 GMT (0308 HK...",2,1,0.3333,0.0638,9,26,14,12,2,47,1,6
3,3,4,https://www.cnn.com/2022/02/24/us/iyw-how-to-h...,How to provide aid to civilians in Ukraine | C...,\n More than a year after Russian forces ...,0,1,-1.0000,0.0147,5,18,9,12,2,68,0,6
4,4,5,https://www.cnn.com/2022/03/07/us/iyw-internat...,International Women's Day 2022: She became a r...,\n It was the middle of the night when Bi...,31,31,0.0000,0.0981,7,22,12,141,2,632,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,130,https://www.cnn.com/2021/01/19/us/give-send-go...,"Proud Boys, right-wing extremists and their su...",\n Before Proud Boys member Nick Ochs was...,32,51,-0.2289,0.1078,12,24,15,188,2,770,2,6
130,130,131,https://www.cnn.com/2021/01/12/us/military-ext...,Military vets were among the Capitol Hill riot...,\n Among the mob of extremists and Trump ...,70,119,-0.2593,0.1165,12,27,16,437,2,1622,12,6
131,131,132,https://www.cnn.com/2021/01/08/us/dc-police-ar...,DC police made far more arrests at height of B...,\n Black Lives Matter protesters in Washi...,19,52,-0.4648,0.0900,14,27,16,210,2,789,4,6
132,132,133,https://www.cnn.com/2021/01/08/us/online-extre...,Extremists intensify calls for violence ahead ...,\n “Trump or war. Today. That simple.”\n ...,58,101,-0.2704,0.1436,12,28,16,309,2,1107,7,6


In [75]:
output_data.to_excel("output_data.xlsx")